# runtime._utils

> TODO fill in description

In [ ]:
#| default_exp runtime._utils

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations

import fbdev
from fbdev.comp.packet import Packet
from fbdev.comp.port import PortSpecCollection, PortCollection, PortID

In [ ]:
#|hide
show_doc(fbdev.runtime._utils.parse_args_into_port_packets)

---

### parse_args_into_port_packets

>      parse_args_into_port_packets
>                                    (port_specs:fbdev.comp.port.PortSpecCollect
>                                    ion, config_vals, signals, *args, **kwargs)

In [ ]:
#|export
def parse_args_into_port_packets(port_specs:PortSpecCollection, config_vals, signals, *args, **kwargs):
    # Fill in input args
    input_vals = {**kwargs}
    
    for port_name, val in zip(port_specs.input.keys(), args):
        if port_name in input_vals: raise ValueError(f"Multiple values provided for '{port_name}'.")
        input_vals[port_name] = val
        
    missing_input_args = set(port_specs.input.keys()) - set(input_vals.keys())
    if len(missing_input_args) > 0:
        raise ValueError(f"Missing values for ports '{missing_input_args}'.")
    
    # Check for unexpected input args
    extra_input_args = set(input_vals.keys()) - set(port_specs.input.keys())
    if len(extra_input_args) > 0:
        raise ValueError(f"Unexpected values for inputs '{extra_input_args}'.")
    
    # Check for unexpected config args
    extra_config_args = set(config_vals.keys()) - set(port_specs.config.keys())
    if len(extra_config_args) > 0:
        raise ValueError(f"Unexpected values for configs '{extra_config_args}'.")
    
    # Check for unexpected signals
    extra_signal_args = set(signals) - set(port_specs.signal.keys())
    if len(extra_signal_args) > 0:
        raise ValueError(f"Unexpected values for signals '{extra_signal_args}'.")
    
    return input_vals, config_vals, signals

In [ ]:
#|hide
show_doc(fbdev.runtime._utils.setup_packet_senders_and_receivers)

---

### setup_packet_senders_and_receivers

>      setup_packet_senders_and_receivers (ports:fbdev.comp.port.PortCollection,
>                                          input_vals, config_vals, *args,
>                                          **kwargs)

In [ ]:
#|export
def setup_packet_senders_and_receivers(ports:PortCollection, input_vals, config_vals, ports_to_get, *args, **kwargs):
    async def packet_sender(port_id: PortID, val):
        await ports[port_id]._put(Packet(val))
    
    async def packet_receiver(port_id: PortID, dest_dict):
        packet = await ports[port_id]._get()
        dest_dict[port_id[1]] = await packet.consume()
        
    output_vals = {}
    message_vals = {}
    
    input_senders = [packet_sender(port.id, input_vals[port.name]) for port in ports.input.values()]
    config_senders = [packet_sender(port.id, config_vals[port.name]) for port in ports.config.values()]
    output_receivers = [packet_receiver(port.id, output_vals) for port in ports.output.values() if port.id in ports_to_get]
    message_receivers = [packet_receiver(port.id, message_vals) for port in ports.message.values() if port.id in ports_to_get]
    
    return output_vals, message_vals, input_senders, config_senders, output_receivers, message_receivers